In [2]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [3]:
#colunas que escolhi para dados escolares
col_escol = ['CODMEC','ANO','TIPOESC','DEP','Qtde_Docentes','NUM_FUNC','MERENDA','CLFUND',
            'ALFUND','TOTALEJA','TOT_EE','TOTAL_ALUNOS','CL_CRECHE','AL_CRECHE','CL_PRE','AL_PRE','CLEM','ALEM']
col_rendi = ['CODMEC','EFABAND9ANOS8S','EFABAND9ANOS9S','EFABAND7S','EFABAND8S','TOTALEF_ABAND','TOTALEF_APROV',
             'TOTALEF_REPROV','TOTAL_EM_ABAND','TOTAL_EM_REPROV','CONCLUINTES_EM','EPABAND','EPREPROV']
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1', usecols = col_escol)
rendimento = pd.read_csv('RendimentoEscolar2000-2015.csv', encoding = 'latin1', usecols = col_rendi)
esc_rmsp = pd.read_csv('ESC2013_RMSP_CEM.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(104292, 18)
Rendimento(14944, 13)
ESC_RMSP(10937, 136)
Nota(1243, 3)


In [5]:
#Existem escolas que estão no dataframe dados_escolares, rendimento, esc_rmsp que eu não tenho o valor da nota dela
#Preciso retirar escolas que não estão no dataframe nota
dados_escolares = dados_escolares[dados_escolares.CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
rendimento = rendimento[rendimento.CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
esc_rmsp = esc_rmsp[esc_rmsp.CODESC.isin(nota['CODIGO DA ENTIDADE'])]

In [6]:
#retirando as colunas em que mais que 50% dos valores são null
dados_escolares1 = dados_escolares.drop(dados_escolares.columns[dados_escolares.isnull().mean()>0.5], axis = 1)
#apagando as linha com valor NaN
dados_escolares1.dropna(inplace = True, how = 'any')

In [7]:
#como eu abri todas as colunas do esc_rmsp, agora vou retirar as que não desejo usar
col_rmsp = ['ID', 'LONGITUDE', 'LATITUDE', 'CODESCTX','NOMEESC', 'NOMEMUN','CODMUN', 'NOMDIST', 'CODDIST', 'COD_SC',
           'BAIRRO','CEP','END_ESC','NUM_ESC','DDD','TELEFONE','ENS_REG','REG_IN_CR','REG_IN_PE','REG_FU_8A',
           'REG_FU_9A','REG_MED','REG_MED_I','REG_MED_N','REG_MED_P','ENS_ESP','ESP_IN_CR','ESP_IN_PE',
           'ESP_FU_8A','ESP_FU_9A','ESP_MED','ESP_MED_I','ESP_MED_N','ESP_MED_P','ESP_EJA_F','ESP_EJA_M','EJA',
           'EJA_FUND','EJA_MED','EJA_PROJ']
esc_rmsp1 = esc_rmsp.drop(col_rmsp, axis = 1)

In [8]:
esc_rmsp1.drop(esc_rmsp1.columns[esc_rmsp1.isnull().mean() > 0.3], axis=1, inplace = True)
esc_rmsp1.dropna(how='any', axis = 0, inplace = True)

In [9]:
print('Dados escolares', dados_escolares1.isnull().sum().sum())
print('Rendimento', rendimento.isnull().sum().sum())
print('ESC_RMSP1', esc_rmsp1.isnull().sum().sum())

Dados escolares 0
Rendimento 0
ESC_RMSP1 0


In [10]:
#primeiro teste vai ser o mais simples e fácil, usar o maior dataset que tenho
teste1 = pd.merge(dados_escolares1, nota, how = 'inner', left_on = 'CODMEC', right_on = 'CODIGO DA ENTIDADE')
teste1.drop(['ANO', 'TIPOESC', 'CODIGO DA ENTIDADE', 'NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
dep = {'Privada': 'privada', 'Estadual':'estadual', 'PRIVADA':'privada', 'ESTADUAL':'estadual',
      'privada':'privada', 'PARTICULAR':'privada','Municipal':'municipal','MUNICIPAL':'municipal',
      'Federal':'federal', 'FEDERAL':'federal'}
teste1.DEP = teste1.DEP.map(dep)
teste1 = pd.get_dummies(teste1, 'DEP')

In [11]:
target = teste1.Classificacao
data = teste1.drop('Classificacao', axis =1)

In [13]:
target.value_counts()

2    7249
3    1355
1    1130
4     656
0       3
Name: Classificacao, dtype: int64

In [11]:
def modelo_linear():
    kf = KFold(n_splits = 5)
    coe = []
    scor = []
    for train_index, test_index in kf.split(data):
        #separando em x,y treino e x,y teste
        x_train, x_test = np.array(data)[train_index], np.array(data)[test_index]
        y_train, y_test = np.array(target)[train_index], np.array(target)[test_index]
        #fit o scaler com base no train, transform train e test
        scaler = StandardScaler()
        scaler.fit(x_train)
        x_train_scaled = scaler.transform(x_train)
        x_test_scaled = scaler.transform(x_test)
        #fit o pca com base no train, transform train e test
        pca = PCA(0.95)
        pca.fit(x_train_scaled)
        x_train_scaled = pca.transform(x_train_scaled)
        x_test = pca.transform(x_test_scaled)
        #fitando o linear regression no train, avaliando no test
        lm = LinearRegression()
        lm.fit(x_train_scaled, y_train)        
        scor.append(lm.score(x_test, y_test))
        coe.append(lm.coef_)
    return scor, coe

In [12]:
modelo_linear()

([0.1884377829856061,
  0.09029241349298468,
  -0.15601777565502362,
  0.18170305307324452,
  -0.0383385878570599],
 [array([ 0.10588923,  0.03667219,  0.04362433,  0.11378414, -0.05950105,
          0.06695005,  0.07821987, -0.02149581]),
  array([ 0.12864273,  0.04836024,  0.00351143,  0.14150184, -0.0147282 ,
          0.06294633,  0.09677731, -0.06857373]),
  array([ 0.14567619, -0.00281191,  0.00208543, -0.1196628 ,  0.05351346,
          0.05477117,  0.06823988, -0.11016167]),
  array([ 0.12778289, -0.07690021, -0.02465833,  0.02082304,  0.08746063,
          0.0120282 ,  0.04965883, -0.04933183]),
  array([ 0.13489566,  0.08708272, -0.00263264, -0.05646346,  0.04594472,
          0.07295263, -0.10952338])])

In [37]:
def modelo_svm(data, target, gamma, C):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    pca = PCA(0.95)
    data = pca.fit_transform(data)
    scores = []
    for g in gamma:
        for c in C:
            svc = SVC(kernel = 'rbf', C=c, gamma = g)
            svc.fit(data,target)
            scores.append((svc.score(data,target), g, c))
            print('Iteração com gamma %f e c %f concluída' %(g, c))
    return scores

In [38]:
gamma = np.linspace(1,5,9)
c = gamma
svm1 = modelo_svm(data, target, gamma, c)

C:\Users\giova\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Iteração com gamma 1.000000 e c 1.000000 concluída
Iteração com gamma 1.000000 e c 1.500000 concluída
Iteração com gamma 1.000000 e c 2.000000 concluída
Iteração com gamma 1.000000 e c 2.500000 concluída
Iteração com gamma 1.000000 e c 3.000000 concluída
Iteração com gamma 1.000000 e c 3.500000 concluída
Iteração com gamma 1.000000 e c 4.000000 concluída
Iteração com gamma 1.000000 e c 4.500000 concluída
Iteração com gamma 1.000000 e c 5.000000 concluída
Iteração com gamma 1.500000 e c 1.000000 concluída
Iteração com gamma 1.500000 e c 1.500000 concluída
Iteração com gamma 1.500000 e c 2.000000 concluída
Iteração com gamma 1.500000 e c 2.500000 concluída
Iteração com gamma 1.500000 e c 3.000000 concluída
Iteração com gamma 1.500000 e c 3.500000 concluída
Iteração com gamma 1.500000 e c 4.000000 concluída
Iteração com gamma 1.500000 e c 4.500000 concluída
Iteração com gamma 1.500000 e c 5.000000 concluída
Iteração com gamma 2.000000 e c 1.000000 concluída
Iteração com gamma 2.000000 e c

In [43]:
svm1.sort()

In [45]:
svm1[-5:]

[(0.8733763109785433, 4.5, 4.5),
 (0.8753969017608005, 5.0, 4.0),
 (0.8762628692389108, 4.5, 5.0),
 (0.8784758972385259, 5.0, 4.5),
 (0.8812662368902145, 5.0, 5.0)]

In [46]:
gamma = np.linspace(5,10,9)
c = gamma
svm1 = modelo_svm(data, target, gamma, c)

C:\Users\giova\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Iteração com gamma 5.000000 e c 5.000000 concluída
Iteração com gamma 5.000000 e c 5.625000 concluída
Iteração com gamma 5.000000 e c 6.250000 concluída
Iteração com gamma 5.000000 e c 6.875000 concluída
Iteração com gamma 5.000000 e c 7.500000 concluída
Iteração com gamma 5.000000 e c 8.125000 concluída
Iteração com gamma 5.000000 e c 8.750000 concluída
Iteração com gamma 5.000000 e c 9.375000 concluída
Iteração com gamma 5.000000 e c 10.000000 concluída
Iteração com gamma 5.625000 e c 5.000000 concluída
Iteração com gamma 5.625000 e c 5.625000 concluída
Iteração com gamma 5.625000 e c 6.250000 concluída
Iteração com gamma 5.625000 e c 6.875000 concluída
Iteração com gamma 5.625000 e c 7.500000 concluída
Iteração com gamma 5.625000 e c 8.125000 concluída
Iteração com gamma 5.625000 e c 8.750000 concluída
Iteração com gamma 5.625000 e c 9.375000 concluída
Iteração com gamma 5.625000 e c 10.000000 concluída
Iteração com gamma 6.250000 e c 5.000000 concluída
Iteração com gamma 6.250000 e

In [48]:
svm1.sort()
svm1[-8:]

[(0.9221591455787549, 8.75, 10.0),
 (0.9224478014047917, 9.375, 8.75),
 (0.9239872991436544, 9.375, 9.375),
 (0.9239872991436544, 10.0, 8.125),
 (0.9253343596651592, 10.0, 8.75),
 (0.9254305782738381, 9.375, 10.0),
 (0.9266814201866641, 10.0, 9.375),
 (0.9276436062734533, 10.0, 10.0)]

In [49]:
gamma = np.linspace(10,15,9)
c = gamma
svm1 = modelo_svm(data, target, gamma, c)

C:\Users\giova\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Iteração com gamma 10.000000 e c 10.000000 concluída
Iteração com gamma 10.000000 e c 10.625000 concluída
Iteração com gamma 10.000000 e c 11.250000 concluída
Iteração com gamma 10.000000 e c 11.875000 concluída
Iteração com gamma 10.000000 e c 12.500000 concluída
Iteração com gamma 10.000000 e c 13.125000 concluída
Iteração com gamma 10.000000 e c 13.750000 concluída
Iteração com gamma 10.000000 e c 14.375000 concluída
Iteração com gamma 10.000000 e c 15.000000 concluída
Iteração com gamma 10.625000 e c 10.000000 concluída
Iteração com gamma 10.625000 e c 10.625000 concluída
Iteração com gamma 10.625000 e c 11.250000 concluída
Iteração com gamma 10.625000 e c 11.875000 concluída
Iteração com gamma 10.625000 e c 12.500000 concluída
Iteração com gamma 10.625000 e c 13.125000 concluída
Iteração com gamma 10.625000 e c 13.750000 concluída
Iteração com gamma 10.625000 e c 14.375000 concluída
Iteração com gamma 10.625000 e c 15.000000 concluída
Iteração com gamma 11.250000 e c 10.000000 con

In [57]:
svm1.sort()
svm1[-8:]

[(0.9474646396613104, 14.375, 13.75),
 (0.9477532954873472, 13.75, 15.0),
 (0.9485230443567786, 15.0, 13.125),
 (0.9488117001828154, 14.375, 14.375),
 (0.9495814490522467, 14.375, 15.0),
 (0.9496776676609257, 15.0, 13.75),
 (0.9507360723563937, 15.0, 14.375),
 (0.951505821225825, 15.0, 15.0)]

In [58]:
def modelo_svm_kf(data, target, gamma, C):
    kf = KFold(n_splits = 5)
    scores = []
    for g in gamma:
            for c in C:
                mean_kf = []
                for train_i, test_i in kf.split(data):
                    #separação dos dados em treino e teste
                    data_train, data_test = data[train_i], data[test_i]
                    target_train, target_test = target[train_i], target[test_i]
                    
                    #normalização dos dados
                    scaler = StandardScaler()
                    scaler.fit(data_train)
                    data_train = scaler.transform(data_train)
                    data_test = scaler.transform(data_test)
                    
                    #pca nos dados
                    pca = PCA(0.95)
                    pca.fit(data_train)
                    data_train = pca.transform(data_train)
                    data_test = pca.transform(data_test)
                    
                    #rodando o svm
                    svc = SVC(kernel = 'rbf', C=c, gamma = g)
                    svc.fit(data_train,target_train)
                    mean_kf.append(svc.score(data_test,target_test))
                                   
                #salvando a média do KFold para g e c dados
                scores.append(((sum(mean_kf)/len(mean_kf)), g, c))
                print('Iteração com gamma %f e c %f concluída' %(g, c))
    return scores

In [59]:
modelo_svm_kf(data, target, [15], [15])

KeyError: '[ 2079  2080  2081 ... 10390 10391 10392] not in index'